In [ ]:
import pandas as pd
from glob import glob
from pyedflib import highlevel as reader
from matplotlib import pyplot as plt
import torch
import pickle
# read an edf file
signals, signal_headers, header = reader.read_edf('/storage/HCI/Sessions/59/Part_1_Trial18_taggingImages1.bdf')

In [ ]:
signals.shape

In [ ]:
signal_labels = [x["label"] for x in signal_headers]
print(signal_labels,f"len:{len(signal_labels)}")
print("=============================")
print(signal_labels.index("EXG1"))
print(signal_labels.index("EXG2"))
print(signal_labels.index("EXG3"))

In [ ]:
signal_headers

In [ ]:
hr_freq = signal_headers[32]["sample_frequency"]
print(hr_freq)
hr_skip = int(hr_freq * 30)
hr_dur = int(hr_freq * 3)
hr_data = signals[33][hr_skip:hr_skip + hr_dur]
plt.plot([i/hr_freq for i in range(len(hr_data))],hr_data)

In [ ]:
event_freq = signal_headers[46]["sample_frequency"]
event_skip = int(event_freq * 29)
event_data = signals[46][event_skip:-event_skip]
plt.plot([i/event_freq for i in range(len(event_data))],event_data)

In [ ]:
timestamps = []
prev = None
for i,v in enumerate(event_data):
    if(not v == prev):
        timestamps.append(i)
        prev = v
timestamps
print(f"stimuate duration: {(timestamps[-2] - timestamps[1])/event_freq}")


In [ ]:
import os
from src.datasets import SessionMeta,RPPG

test = SessionMeta("./datasets/hci/Sessions/521/",cropped_dir="./datasets/hci/cropped_faces/521/")
test.__dict__

In [ ]:
c = RPPG.get_default_config()
c.clip_duration = 10
c.num_frames = c.clip_duration * 5
c.train_ratio = 1.0
# c.meta_folder=None

In [ ]:
# from src.models import Detector
# import pandas as pd
# import torch
# from src.models import Detector
# from main import get_config,init_accelerator,set_seed
# from torchinfo import summary

# config = get_config("./configs/rppg.yml")

# # initialize accelerator and trackers (if enabled)
# accelerator = init_accelerator(config)
# model = Detector(config.model, 50, accelerator)

In [ ]:
x = RPPG(c)

In [ ]:
# x.save_meta()

In [ ]:
len(x.session_metas)

In [ ]:
r = x.get_dict(100)
frames,label,hr_data, masks, measures,wd = r["frames"],r["label"],r["hr_data"],r["mask"],r["measures"],r["wd"]

In [15]:
frames.shape

torch.Size([50, 3, 150, 150])

In [17]:
import torchvision.transforms as T
trans =  T.Compose([
    T.Resize(224, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(224),
    T.ConvertImageDtype(torch.float32),
    T.Normalize((0.48145466, 0.4578275, 0.40821073),
                (0.26862954, 0.26130258, 0.27577711)),
])
trans(frames)

tensor([[[[-0.9456, -0.9164, -0.9164,  ..., -1.1353, -0.7704, -0.4784],
          [-1.0039, -0.9164, -0.9018,  ..., -1.2083, -0.9164, -0.6682],
          [-0.9018, -0.8434, -0.8580,  ..., -1.2813, -1.0915, -0.9310],
          ...,
          [-1.3397, -1.3105, -1.2667,  ..., -1.2083, -1.1791, -1.1353],
          [-1.3543, -1.2959, -1.2521,  ..., -1.2229, -1.1937, -1.1353],
          [-1.3543, -1.2959, -1.2375,  ..., -1.2229, -1.1791, -1.1207]],

         [[-0.8666, -0.8366, -0.8366,  ..., -1.2118, -0.7766, -0.4314],
          [-0.9267, -0.8366, -0.8216,  ..., -1.2869, -0.9267, -0.6415],
          [-0.8216, -0.7616, -0.7766,  ..., -1.3469, -1.1218, -0.9267],
          ...,
          [-1.3769, -1.3469, -1.3019,  ..., -1.2418, -1.2118, -1.1668],
          [-1.3919, -1.3319, -1.2869,  ..., -1.2568, -1.2268, -1.1668],
          [-1.3919, -1.3319, -1.2718,  ..., -1.2568, -1.2118, -1.1518]],

         [[-0.7834, -0.7408, -0.7408,  ..., -1.0821, -0.6839, -0.3711],
          [-0.8403, -0.7550, -

In [ ]:
measures["bpm"]

In [ ]:
masks

In [ ]:
import numpy as np
t = np.stack(frames[:30])

In [ ]:
print(len(hr_data),len(frames))

In [ ]:
plt.imshow(frames[0])

In [ ]:
plt.imshow(np.stack(frames[:30],axis=1).reshape((150,-1,3)))

In [ ]:
plt.plot(hr_data)

In [ ]:
import heartpy as hp
hp.plotter(wd, measures)
#display computed measures
for measure in measures.keys():
    print('%s: %f' %(measure, measures[measure]))

In [ ]:
from tqdm import tqdm
for i in tqdm(range(len(x))):
    try:
        x[i]
    except Exception as e:
        print(i,e)

In [ ]:
import math
from src.datasets import resample
self = x
idx = 199

session_idx =  next(i for i,x in enumerate(self.stack_session_clips) if  idx < x)
session_meta = self.session_metas[session_idx]
session_offset_duration =  (idx - (0 if session_idx == 0 else self.stack_session_clips[session_idx-1]))*self.clip_duration
# heart rate data processing
signals, signal_headers, _ = reader.read_edf(session_meta.bdf_path,ch_names=["EXG1","EXG2","EXG3","Status"])
_hr_datas = []
for hr_channel_idx in range(3):
    try:
        assert  int(session_meta.session_hr_sample_freq) == int(signal_headers[hr_channel_idx]["sample_frequency"])
        # - the ERG sample frequency
        hr_sample_freq =  session_meta.session_hr_sample_freq
        # - the amount of samples to skip, including the 30s stimulation offset and session clip offset.
        hr_sample_offset =  session_meta.flag_hr_beg_sample + int(session_offset_duration*hr_sample_freq)
        # - the amount of samples for the duration of a clip
        hr_clip_samples = int(hr_sample_freq*self.clip_duration)
        # - fetch heart rate data of clip duration
        _hr_data = signals[hr_channel_idx][hr_sample_offset:hr_sample_offset + hr_clip_samples]
        # - preprocess the ERG data: filter out the noise.
        _hr_data = hp.filter_signal(_hr_data, cutoff = 0.05, sample_rate = session_meta.session_hr_sample_freq, filtertype='notch')
        # - scale down the ERG value to 3.4 max.
        _hr_data = (_hr_data - _hr_data.min())/(_hr_data.max()-_hr_data.min()) * 3.4
        # - resample the ERG
        _hr_data = resample(_hr_data, len(_hr_data) * 10)
        # - process the ERG data: get measurements.
        _wd, _measures = hp.process(hp.scale_data(_hr_data),session_meta.session_hr_sample_freq * 10)
        # - nan/error check
        # if(_measures["bpm"] > 180 or _measures["bpm"] < 41):
            # continue
    
        for v in _measures.values():
            # ignore
            if type(v)==float and math.isnan(v):
                break
        else:
            # - save for comparison.
            _hr_datas.append((_hr_data,_measures,_wd))
    except Exception as e:
        print(f"Error occur during heart rate analysis for index {idx}:{e}")
        continue

if(len(_hr_datas) == 0):
    raise Exception(f"Unable to process the ERG data for index {idx}")

# get the best ERG measurement result with the sdnn
best_pair = sorted(_hr_datas,key=lambda x : x[1]["sdnn"])[0]
hr_data,measures,wd = best_pair[0], best_pair[1], best_pair[2]
bpm = measures["bpm"]
1/(pow(2*math.pi,0.5))*pow(math.e,(-pow((31-(72-41)),2)/2))
label = torch.tensor([1/(pow(2*math.pi,0.5))*pow(math.e,(-pow((k-(bpm-41)),2)/2)) for k in range(180)])

import heartpy as hp
hp.plotter(wd, measures)
#display computed measures
for measure in measures.keys():
    print('%s: %f' %(measure, measures[measure]))
